In [21]:
import numpy as np
import functools as ft
import networkx as nx

In [22]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def p(d):
    return np.random.shuffle(np.eye(d))


def inverse(P):
    return np.linalg.inv(P)


def permute(P, H):
    return P.dot(H)


def cosine_similarity(A, B):
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return np.dot(A, B) / (norm_A * norm_B)

In [23]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: cosine_similarity(V, x[1]))

In [24]:
def initVertices(G, d=10000):
    for n in G:
        G.nodes[n]["hdv"] = hdv(d)

In [25]:
def initNodeMem(G, memory):
    for n in G:
        xs = map(lambda x: G.nodes[x[1]]["hdv"], G.edges(n))
        G.nodes[n]["mem"] = bundle(xs)
        memory.addVector(f"mem{n}", G.nodes[n]["mem"])

In [26]:
def initGraph(G, memory):
    Gh = []
    for n in G.nodes():
        Gh.append(bind([G.nodes[n]["hdv"], G.nodes[n]["mem"]]))
    # Gh = bundle(Gh) / 2
    Gh = bundle(Gh) / 2
    memory.addVector("g", Gh)
    return Gh

In [27]:
G = nx.gnm_random_graph(30, 150)
memory = ItemMemory()

initVertices(G, 1000)
initNodeMem(G, memory)
Gh = initGraph(G, memory)

In [36]:
query = bind([Gh, G.nodes[6]["hdv"]])
res = memory.cleanup(query)
print(res[0])
print(cosine_similarity(res[1], query))

mem6
0.31862185542238414


In [37]:
# nx.draw(G)